In [1]:
%reset -f

In [2]:
import pandas as pd
import numpy as np
import os
import sys

In [3]:
# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [4]:
os.getcwd()

'C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data\\notebooks'

In [5]:
os.chdir("..")

In [6]:
os.getcwd()

'C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data'

In [7]:
os.listdir()

['.virtual_documents',
 'Community',
 'data_processed',
 'data_raw',
 'desktop.ini',
 'notebooks',
 'site']

In [8]:
os.getcwd()

'C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data'

In [9]:
# -----------------------------
# 2. Sample Data Preparation
# -----------------------------
os.chdir("C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data\\data_processed\\Pickel files")
df_2020_2025 = pd.read_pickle("df_2020_2025.pkl")

In [10]:
os.chdir("C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data")

In [11]:
os.getcwd()

'C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data'

#### Data import

In [12]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from datetime import datetime

In [13]:
df_2020_2025.shape

(36212597, 13)

In [14]:
df_2020_2025.head(2)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,A955EE529EC6DF27,classic_bike,2020-12-23 16:31:04.165,2020-12-23 16:45:28.611,Forsyth St & Broome St,5453.05,E 33 St & 1 Ave,6197.08,40.718939,-73.992663,40.743227,-73.974498,member
1,E42C1EDE5F256E87,classic_bike,2020-09-14 18:34:08.324,2020-09-14 18:36:07.759,1 Ave & E 18 St,5854.09,E 20 St & FDR Drive,5886.02,40.733812,-73.980544,40.733143,-73.975739,member


#### 0. Packed bubble chart

In [15]:
os.getcwd()

'C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data'

In [16]:
# df_2020_2025 = df_2020_2025[:1000]

In [17]:
df_2020_2025.shape

(36212597, 13)

In [18]:
os.getcwd()

'C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data'

In [19]:
os.chdir("C:\\Users\\rajpprit\\Documents\\UK I&I\\Trainings\\Vizcon 2025\\New York city bike data")

In [ ]:
os.getcwd()

In [24]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import math

# Convert started_at to datetime first
df_2020_2025['started_at'] = pd.to_datetime(df_2020_2025['started_at'])

# Create year and month columns
df_2020_2025['year'] = df_2020_2025['started_at'].dt.year.astype(int)
df_2020_2025['month'] = df_2020_2025['started_at'].dt.month.astype(int)

# Print unique years to debug
print("Unique years in original data:", sorted(df_2020_2025['year'].unique()))

# Perform the groupby operation
monthly_counts = df_2020_2025.groupby(['year', 'month']).size().reset_index(name='count')

# Print unique years after groupby
print("Unique years after groupby:", sorted(monthly_counts['year'].unique()))

# Create month names for labels
month_names = {
    1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
    7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'
}
monthly_counts['month_name'] = monthly_counts['month'].map(month_names)
monthly_counts['label'] = monthly_counts['month_name'] + ' ' + monthly_counts['year'].astype(str)

# Create color mapping for years with colors matching the map theme
unique_years = sorted(monthly_counts['year'].unique())
colors = [
    'rgba(76, 114, 176, 0.8)',   # Subtle blue
    'rgba(221, 132, 82, 0.8)',   # Subtle orange
    'rgba(85, 168, 104, 0.8)',   # Subtle green
    'rgba(129, 114, 179, 0.8)',  # Subtle purple
    'rgba(147, 120, 96, 0.8)',   # Subtle brown
    'rgba(180, 151, 231, 0.8)'   # Subtle lavender
]

# Ensure we have enough colors
while len(colors) < len(unique_years):
    colors.extend(colors)

# Create year_colors dictionary
year_colors = dict(zip(unique_years, colors[:len(unique_years)]))

print("Year-color mapping:", year_colors)

# Function to calculate bubble positions in a spiral layout
def get_spiral_positions(n, spacing=0.5):
    positions = []
    a = spacing  # Controls spacing between points
    b = spacing  # Controls spacing between rings
    for i in range(n):
        t = np.sqrt(i) * 2 * np.pi
        x = (a * t) * np.cos(t)
        y = (b * t) * np.sin(t)
        positions.append((x, y))
    return positions

# Create figure
fig = go.Figure()

# Sort data by count to place larger bubbles first
monthly_counts = monthly_counts.sort_values('count', ascending=False)

# Calculate positions for all bubbles
all_positions = get_spiral_positions(len(monthly_counts))
position_index = 0

for year in unique_years:
    year_data = monthly_counts[monthly_counts['year'] == year]
    
    # Calculate scaling factor based on counts
    max_count = monthly_counts['count'].max()
    scale_factor = 50 / np.sqrt(max_count)  # Adjust this value to change bubble sizes
    
    for _, row in year_data.iterrows():
        x, y = all_positions[position_index]
        position_index += 1
        
        fig.add_trace(go.Scatter(
            x=[x],
            y=[y],
            mode='markers+text',
            marker=dict(
                size=np.sqrt(row['count']) * scale_factor,
                color=year_colors[row['year']],
                line=dict(color='rgba(255,255,255,0.3)', width=1)
            ),
            text=f"{row['month_name']} {row['year']}<br>{row['count']:,} rides",
            textposition="middle center",
            name=str(year),
            hovertemplate="%{text}<extra></extra>",
            showlegend=True if row.name == year_data.index[0] else False
        ))

# Update layout
fig.update_layout(
    title={
        'text': "Monthly Bike Rides Distribution (2020-2025)",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(color='white', size=24)
    },
    showlegend=True,
    legend_title_text="Year",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99,
        font=dict(color='white'),
        title=dict(font=dict(color='white'))
    ),
    paper_bgcolor='#1a1f35',
    plot_bgcolor='#1a1f35',
    xaxis=dict(
        showgrid=False,
        showticklabels=False,
        zeroline=False,
    ),
    yaxis=dict(
        showgrid=False,
        showticklabels=False,
        zeroline=False,
        scaleanchor="x",
        scaleratio=1
    ),
    width=1000,
    height=800
)

# Save the figure
fig.write_html("monthly_packed_bubbles.html")


Unique years in original data: [np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]
Unique years after groupby: [np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]
Year-color mapping: {np.int64(2019): 'rgba(76, 114, 176, 0.8)', np.int64(2020): 'rgba(221, 132, 82, 0.8)', np.int64(2021): 'rgba(85, 168, 104, 0.8)', np.int64(2022): 'rgba(129, 114, 179, 0.8)', np.int64(2023): 'rgba(147, 120, 96, 0.8)', np.int64(2024): 'rgba(180, 151, 231, 0.8)', np.int64(2025): 'rgba(76, 114, 176, 0.8)'}
